In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
root = '../input/ashrae-energy-prediction/'
train_df = pd.read_csv(root + 'train.csv')
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"], format='%Y-%m-%d %H:%M:%S')

weather_train_df = pd.read_csv(root + 'weather_train.csv')
test_df = pd.read_csv(root + 'test.csv')
weather_test_df = pd.read_csv(root + 'weather_test.csv')
building_meta_df = pd.read_csv(root + 'building_metadata.csv')
sample_submission = pd.read_csv(root + 'sample_submission.csv')

<H2> Discovering data  roughly

In [ ]:
import seaborn as sns

<H2> NaNs 

In [ ]:
sns.heatmap(building_meta_df.isnull(), cbar=False)

In [ ]:
sns.heatmap(weather_train_df.isnull(), cbar=False)

<H2>Which site has how many building ? 

In [ ]:
sitelist = list(building_meta_df.site_id.unique())
SitePareto = [len(building_meta_df[building_meta_df.site_id == i].building_id.unique()) for i in sitelist ]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from matplotlib import rcParams

In [ ]:
SitePareto = pd.DataFrame(SitePareto)
SitePareto.columns = ["BuildingQuantity"]
SitePareto.index = ['Site {}'.format(i)  for i in range(16)]
SitePareto = SitePareto.sort_values(by="BuildingQuantity",ascending=False)
SitePareto["cumpercentage"] = SitePareto["BuildingQuantity"].cumsum()/SitePareto["BuildingQuantity"].sum()*100

In [ ]:
rcParams['figure.figsize'] = 15, 5
fig, ax = plt.subplots()
ax.bar(SitePareto.index, SitePareto["BuildingQuantity"], color="C0")
ax2 = ax.twinx()
ax2.plot(SitePareto.index, SitePareto["cumpercentage"], color="C1", marker="D", ms=7)
ax2.yaxis.set_major_formatter(PercentFormatter())
ax.tick_params(axis="y", colors="C0")
ax2.tick_params(axis="y", colors="C1")

<H2> Site 3 has much more buildings than rest. Therefore, i will analyze it first.

<h2>Corr plot

In [ ]:
corr = weather_train_df[weather_train_df.site_id == 13].corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,square=True, linewidths=.5, cbar_kws={"shrink": .5})

<li> sea level pressure variance increases, when air pressure and evaporation temperature decrease.

In [ ]:
rcParams['figure.figsize'] = 12, 8
import matplotlib.dates as mdates
data=weather_train_df[weather_train_df.site_id ==13]
data=data.reset_index(drop=True)
years = mdates.YearLocator()   
fig, ax = plt.subplots()
ax.plot('timestamp', 'dew_temperature', data=data,color="k")
plt.legend()
ax.plot('timestamp', 'air_temperature', data=data,color="g",alpha=0.7)
plt.legend()
ax2 = ax.twinx()
ax2.plot('timestamp', 'sea_level_pressure', data=data,color="r",alpha=0.3)
ax.xaxis.set_major_locator(years)
plt.legend()

In [ ]:
#Visualization Function 
def polar(a):
    df = weather_train_df[weather_train_df.site_id ==a]
    df = df.reset_index(drop=True)
    N = 360
    polarDataRad=[(i*np.pi)/180  for i in range (360)]
    polarDataQua=[len(df[df["wind_direction"]==i])  for i in range (360)]
    polarDataSpeed=[np.mean(df[df["wind_direction"]==i]["wind_speed"])  for i in range (360)]
    polarDataAirTemp=[np.mean(df[df["wind_direction"]==i]["air_temperature"])  for i in range (360)]
    width = [0.2 for i in range(360)]
    DataRad = np.array(polarDataRad)
    DataQua = np.array(polarDataQua)
    Dataspeed = np.array(polarDataSpeed)
    DataTemp = np.array(polarDataAirTemp)
    width = np.array(width)
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, subplot_kw=dict(projection='polar'))
    ax1.bar( DataRad, DataQua, width=width, bottom=0.0,  alpha=0.7 )
    ax2.bar(DataRad,  Dataspeed, width=width, bottom=0.0,  alpha=0.7,color="r")
    ax3.bar(DataRad,  DataTemp, width=width, bottom=-0.1 , alpha=0.7,color="k")
    plt.tight_layout()
    plt.show()

<H2> Wind Direction distribution & Wind Speed vs Wind Direction & Air Temperature vs Wind Direction

In [ ]:
polar(3) # will be used for sites geographical orientation

<H2>  Rainfall (cm**3/m2) comparison  between sites


<li> first, Site based NaN check 

In [ ]:
for i in range(16):
    print("Site İd:{} have {}/{} % {} missing vaule for precipitation".format(i,weather_train_df[weather_train_df.site_id == i].precip_depth_1_hr.isnull().sum(),
                                                                      len(weather_train_df[weather_train_df.site_id == i].precip_depth_1_hr),
                                                                      np.round((weather_train_df[weather_train_df.site_id == i].precip_depth_1_hr.isnull().sum()/
                                                                      len(weather_train_df[weather_train_df.site_id == i].precip_depth_1_hr))*100)                                                                                                                                          
                                                                     ))

<H3> Site id 1 , 5 , 7 , 11 , 12 , 15 do not have sufficent data for Precipitation. 